In [6]:
from inspect import BoundArguments
import sys
from mip import *
from collections import deque

In [45]:


def leitura(file):
    f = open(file,'r')
    res_var = f.readline()
    nb_var,nb_res= int(res_var.split()[0]), int(res_var.split()[1])
    
    l = f.readline()
    max_objetivo = [float(m) for m in l.split()]

    const_restricoes = []
    for i in range(nb_res):
        l = f.readline()
        const_restricoes.append([float(j) for j in l.split()] )
    
    restricao = []
    for k in range(nb_res):
        restricao.append(const_restricoes[k][-1])
        del(const_restricoes[k][-1])

    f.close()

    return  nb_var, nb_res , max_objetivo , const_restricoes , restricao 

def criaModelo(nb_var, nb_res , max_objetivo , const_restricoes , restricao ):
    modelo = Model("teste")
    x = [modelo.add_var(var_type="CONTINUOUS", lb=0.0, name="x" + str(i))
         for i in range(nb_var)]
    
    modelo.objective =maximize(xsum(max_objetivo[i]*x[i] for i in range(nb_var)))
    
    for j in range(nb_res):
        modelo += xsum(const_restricoes[j][i]* x[i] for i in range(nb_var)) <= restricao[j], "Restri" + str(j)
    
    return modelo
    

def branchAndBound(sol_x, nb_res , const_restricoes , restricao,fim):
    x = []
    for i in range(len(sol_x)):
        x_inteiro = int(sol_x[i])
        if sol_x[i] != x_inteiro :
            x.append(sol_x[i]-0.5)
            
    var_proxima = min(x)
    var_proxima+= 0.5
    ramo_menor = int(var_proxima)
    ramo_maior = ramo_menor+1
    nconst_restri= []
    for j in range(len(sol_x)):
        nconst_restri.append(0)
        if (sol_x[j]== var_proxima):
            nconst_restri[j] = 1
    
    const_restricoes.append(nconst_restri)
    nb_res+= 1
    restricao.append(ramo_menor)
    fim+=1
    return  nb_res , const_restricoes , restricao , fim
    
    
nb_var, nb_res , max_objetivo , const_restricoes , restricao = leitura("exemplo1.txt")
fim = 1
sol_x = []
while fim <=2 :
    modelo = criaModelo(nb_var, nb_res , max_objetivo , const_restricoes , restricao)
    status = modelo.optimize()
    print(f"status ",status)
    print("solução:",modelo.objective_value)
    print("Solution:")
    for v in modelo.vars:
        sol_x.append(v.x)
        print(v.name, " = ", v.x)
    nb_res ,const_restricoes , restricao,fim = branchAndBound(sol_x,nb_res ,const_restricoes , restricao,fim)
    



status  OptimizationStatus.OPTIMAL
solução: 15.666666666666668
Solution:
x0  =  0.0
x1  =  0.8333333333333336
x2  =  0.9166666666666664
status  OptimizationStatus.OPTIMAL
solução: 14.0
Solution:
x0  =  0.0
x1  =  0.0
x2  =  1.75


In [3]:
2+3

5